In [18]:
from sklearn.svm import LinearSVC
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import collections
import os
import seaborn
seaborn.set()
from sklearn.externals import joblib

In [19]:
svc = joblib.load('svc.pkl')
encoded = np.load('./npy_data/data_encoded_d.npy')
varvals = np.load('./npy_data/varvals_d.npy')
pathdataOH = np.load("./npy_data/path_data_oh_d.npy")
oldpath = np.load('./npy_data/old_path_d.npy')

In [20]:
varvals

array([2., 2., 3., ..., 3., 2., 3.])

In [21]:
varvals[20251600]

3.0

In [22]:
varvals[20251599]

2.0

In [23]:
varvals[6074457]

4.0

In [32]:
oldpath[20251600]

12016733

In [25]:
maxCoef = np.absolute(svc.coef_).max()

idxM = np.argmax(np.absolute(svc.coef_))
numnz = np.nonzero(svc.coef_)[1].shape
idxNZus = np.nonzero(svc.coef_)[1]
coefs = svc.coef_[0,:]

nnzcoefs = coefs[idxNZus]

idxSort = np.argsort(np.absolute(nnzcoefs))
idxSort = np.flipud(idxSort)
idxNZ = idxNZus[idxSort]

coefPaths = pathdataOH[idxNZ]

tile_path = np.trunc(coefPaths/(16**5))
tile_step = np.trunc((coefPaths - tile_path*16**5)/2)
tile_phase = np.trunc((coefPaths- tile_path*16**5 - 2*tile_step))

In [29]:
print(tile_path, tile_step, tile_phase)

(array([0., 0., 0.]), array([30., 30., 30.]), array([0., 0., 0.]))


In [10]:
vhex = np.vectorize(hex)

In [11]:
vectorizedPath = vhex(tile_path.astype('int'))
vectorizedStep = vhex(tile_step.astype('int'))
vectorizedPhase = vhex(tile_phase.astype('int'))

In [30]:
tile_loc = np.column_stack((tile_path, tile_step))
print(tile_loc)
print(nnzcoefs[idxSort])
print(oldpath[idxNZ])
print(varvals[idxNZ])

[[ 0. 30.]
 [ 0. 30.]
 [ 0. 30.]]
[ 0.6158557  -0.49575757 -0.0089541 ]
[12016733 12016733  3648813]
[3. 2. 4.]


In [13]:
def tileSearch(arg):
    vecpath = str(vectorizedPath[int(arg)])
    vecpath = vecpath[2:].zfill(4)
    try:
        proc = subprocess.check_output("cat " + args.assembly_fwi + " | grep :" + vecpath, shell=True)
        return proc
    except CalledProcessError as e:
        return "Assembly index file not found or `cat` command not available. Continuing..."
    

In [14]:
tileSearch(12559433)

IndexError: index 12559433 is out of bounds for axis 0 with size 3